# Finjustering av Open AI-modeller

Denna anteckningsbok är baserad på den aktuella vägledningen som tillhandahålls i [Finjustering](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dokumentationen från Open AI.

Finjustering förbättrar prestandan hos grundmodeller för din applikation genom att träna om den med ytterligare data och kontext som är relevant för just det specifika användningsfallet eller scenariot. Observera att prompttekniker som _few shot learning_ och _retrieval augmented generation_ låter dig förbättra standardprompten med relevant data för att höja kvaliteten. Dessa metoder är dock begränsade av max token-fönstrets storlek för den riktade grundmodellen.

Med finjustering tränar vi effektivt om modellen själv med den nödvändiga datan (vilket tillåter oss att använda många fler exempel än vad som får plats i max token-fönstret) - och distribuerar en _anpassad_ version av modellen som inte längre behöver få exempel vid inferenstid. Detta förbättrar inte bara effektiviteten i vår promptdesign (vi har mer flexibilitet att använda token-fönstret för andra saker) utan kan också potentiellt förbättra våra kostnader (genom att minska antalet tokens vi behöver skicka till modellen vid inferenstid).

Finjustering har 4 steg:
1. Förbered träningsdata och ladda upp den.
1. Kör träningsjobbet för att få en finjusterad modell.
1. Utvärdera den finjusterade modellen och iterera för kvalitet.
1. Distribuera den finjusterade modellen för inferens när du är nöjd.

Observera att inte alla grundmodeller stödjer finjustering - [kolla OpenAI-dokumentationen](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) för den senaste informationen. Du kan också finjustera en tidigare finjusterad modell. I denna handledning kommer vi att använda `gpt-35-turbo` som vår målgrundmodell för finjustering.

---


### Steg 1.1: Förbered din dataset

Låt oss bygga en chatbot som hjälper dig att förstå det periodiska systemet genom att svara på frågor om ett grundämne med en limerick. I _denna_ enkla handledning kommer vi bara att skapa en dataset för att träna modellen med några exempel på svar som visar det förväntade formatet på datan. I ett verkligt användningsfall skulle du behöva skapa en dataset med många fler exempel. Du kan också eventuellt använda en öppen dataset (för ditt applikationsområde) om en sådan finns, och omformatera den för användning vid finjustering.

Eftersom vi fokuserar på `gpt-35-turbo` och söker ett svar i en enda vändning (chat completion) kan vi skapa exempel med [detta föreslagna format](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) som speglar OpenAI:s krav för chat completion. Om du förväntar dig innehåll i flera vändningar skulle du använda [formatet för exempel med flera vändningar](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) som inkluderar en `weight`-parameter för att signalera vilka meddelanden som ska användas (eller inte) i finjusteringsprocessen.

Vi kommer att använda det enklare formatet för en enda vändning för vår handledning här. Datan är i [jsonl-format](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) med 1 post per rad, där varje post representeras som ett JSON-formaterat objekt. Utdraget nedan visar 2 poster som exempel – se [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) för hela exempeluppsättningen (10 exempel) som vi använder för vår finjusteringshandledning. **Notera:** Varje post _måste_ definieras på en enda rad (inte uppdelad över flera rader som är vanligt i en formaterad JSON-fil)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

I ett verkligt användningsfall behöver du en mycket större uppsättning exempel för bra resultat – avvägningen kommer att vara mellan kvaliteten på svaren och tiden/kostnaderna för finjustering. Vi använder en liten uppsättning så att vi snabbt kan slutföra finjusteringen för att illustrera processen. Se [detta exempel från OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) för en mer komplex handledning i finjustering.


---

### Steg 1.2 Ladda upp din dataset

Ladda upp data med hjälp av Files API [som beskrivs här](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Observera att för att köra denna kod måste du först ha gjort följande steg:
 - Installerat Python-paketet `openai` (se till att du använder en version >=0.28.0 för de senaste funktionerna)
 - Satt miljövariabeln `OPENAI_API_KEY` till din OpenAI API-nyckel
För att lära dig mer, se [Setup-guiden](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) som tillhandahålls för kursen.

Kör nu koden för att skapa en fil för uppladdning från din lokala JSONL-fil.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Steg 2.1: Skapa finjusteringsjobbet med SDK:n


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Steg 2.2: Kontrollera status för jobbet

Här är några saker du kan göra med `client.fine_tuning.jobs` API:
- `client.fine_tuning.jobs.list(limit=<n>)` - Lista de senaste n finjusteringsjobben
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Hämta detaljer för ett specifikt finjusteringsjobb
- `client.fine_tuning.jobs.cancel(<job_id>)` - Avbryt ett finjusteringsjobb
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Lista upp till n händelser från jobbet
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Det första steget i processen är _att validera träningsfilen_ för att säkerställa att data är i rätt format.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Steg 2.3: Spåra händelser för att övervaka framsteg


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Steg 2.4: Visa status i OpenAI-instrumentpanelen


Du kan också se statusen genom att besöka OpenAI:s webbplats och utforska avsnittet _Fine-tuning_ på plattformen. Detta visar statusen för det aktuella jobbet och låter dig även följa historiken för tidigare körningar av jobb. I denna skärmdump kan du se att den tidigare körningen misslyckades, och den andra körningen lyckades. För kontext hände detta när den första körningen använde en JSON-fil med felaktigt formaterade poster – när detta rättades till slutfördes den andra körningen framgångsrikt och gjorde modellen tillgänglig för användning.

![Fine-tuning job status](../../../../../translated_images/sv/fine-tuned-model-status.563271727bf7bfba.png)


Du kan också se statusmeddelanden och mätvärden genom att scrolla längre ner i den visuella instrumentpanelen som visas:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/sv/fine-tuned-messages-panel.4ed0c2da5ea1313b.png) |  ![Metrics](../../../../../translated_images/sv/fine-tuned-metrics-panel.700d7e4995a65229.png)|


---

### Steg 3.1: Hämta ID & Testa finjusterad modell i kod


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Steg 3.2: Ladda och testa finjusterad modell i Playground

Du kan nu testa den finjusterade modellen på två sätt. Först kan du besöka Playground och använda rullgardinsmenyn Models för att välja din nyfinjusterade modell från de listade alternativen. Det andra alternativet är att använda "Playground"-alternativet som visas i Fine-tuning-panelen (se skärmdump ovan) vilket startar följande _jämförande_ vy som visar grundmodellen och den finjusterade modellversionen sida vid sida för snabb utvärdering.

![Fine-tuning job status](../../../../../translated_images/sv/fine-tuned-playground-compare.56e06f0ad8922016.png)

Fyll helt enkelt i systemkontexten som användes i din träningsdata och ange din testfråga. Du kommer att märka att båda sidor uppdateras med identisk kontext och fråga. Kör jämförelsen och du kommer att se skillnaden i svaren mellan dem. _Notera hur den finjusterade modellen återger svaret i det format du angav i dina exempel medan grundmodellen helt enkelt följer systemprompten_.

![Fine-tuning job status](../../../../../translated_images/sv/fine-tuned-playground-launch.5a26495c983c6350.png)

Du kommer att märka att jämförelsen också visar tokenräkningen för varje modell och den tid som tagits för inferensen. **Detta specifika exempel är ett förenklat sådant som är avsett att visa processen men speglar inte en verklig dataset eller scenario**. Du kan märka att båda exemplen visar samma antal tokens (systemkontext och användarprompt är identiska) med den finjusterade modellen som tar längre tid för inferens (anpassad modell).

I verkliga scenarier kommer du inte att använda ett leksaksexempel som detta, utan finjustera mot verklig data (t.ex. produktkatalog för kundservice) där kvaliteten på svaret kommer att vara mycket tydligare. I _det_ sammanhanget kommer det att krävas mer anpassad promptteknik för att få motsvarande svarskvalitet med grundmodellen, vilket ökar tokenanvändningen och potentiellt den relaterade bearbetningstiden för inferens. _För att prova detta, kolla in finjusteringsexemplen i OpenAI Cookbook för att komma igång._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfriskrivning**:
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, vänligen observera att automatiska översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess modersmål bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för några missförstånd eller feltolkningar som uppstår till följd av användningen av denna översättning.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
